In [1]:
# write and read from the FSM and compare with LabVIEW vi
import nidaqmx
import numpy as np

scaling = 1.00457
conversion = {'x': 8.58/scaling, 'y': 6.22/scaling}

def microm_to_volts(pos_in_microm, axis):
    return pos_in_microm / conversion[axis]

def volts_to_microm(pos_in_volts, axis):
    return pos_in_volts * conversion[axis]


x_terminal = '/Dev1/ai1'
y_terminal = '/Dev1/ai0'
with nidaqmx.Task() as x_task:
    x_task.ai_channels.add_ai_voltage_chan(x_terminal)
    curr_pos = x_task.read(number_of_samples_per_channel=8)
    curr_pos = np.mean(curr_pos)
    print('x: volts:', curr_pos, 'um', volts_to_microm(curr_pos, 'x'))
    
with nidaqmx.Task() as y_task:
    y_task.ai_channels.add_ai_voltage_chan(y_terminal)
    curr_pos = y_task.read(number_of_samples_per_channel=8)
    curr_pos = np.mean(curr_pos)
    print('y: volts:', curr_pos, 'um', volts_to_microm(curr_pos, 'y'))


x: volts: -0.3596562955127338 um -3.071812830862216
y: volts: 0.0463080256290676 um 0.28672558349622274


In [2]:
import time
# go to a position --> use smooth sinusoidal sweeping from start to end position
x_terminal_out = '/Dev1/ao1'
y_terminal_out = '/Dev1/ao0'
new_y = 1   # um
scan_freq = 1000
n_pts = 200

# read current position
with nidaqmx.Task() as y_task:
    y_task.ai_channels.add_ai_voltage_chan(y_terminal)
    curr_pos = y_task.read()
    print('Old y: volts:', curr_pos, 'um', volts_to_microm(curr_pos, 'y'))

# make sine-like profile for sweeping the values from one position to the next
delta_pos = microm_to_volts(new_y, 'y') - curr_pos
x_sine = np.linspace(-np.pi/2, np.pi/2, n_pts)
vsweep_array = (np.sin(x_sine)+1)/2*delta_pos + curr_pos

# scan the FSM until the desired position
with nidaqmx.Task() as task:
    output_volt = task.ao_channels.add_ao_voltage_chan(y_terminal_out, 'FSM y axis', min_val=-3.5, max_val=3.5)
    #output_volt.ao_term_cfg = nidaqmx.constants.TerminalConfiguration.DEFAULT
    
    task.timing.samp_timing_type = nidaqmx.constants.SampleTimingType.SAMPLE_CLOCK
    task.timing.samp_quant_samp_mode = nidaqmx.constants.AcquisitionType.FINITE
    task.timing.samp_quant_samp_per_channel = n_pts
    task.timing.samp_clk_rate = scan_freq
    task.timing.samp_clk_active_edge = nidaqmx.constants.Edge.RISING
    #task.timing.samp_clk_src = 
    
    task.write(vsweep_array, auto_start=True, timeout=5)
    task.wait_until_done(timeout=5)

# read the new position
with nidaqmx.Task() as y_task:
    y_task.ai_channels.add_ai_voltage_chan(y_terminal)
    curr_pos = y_task.read(number_of_samples_per_channel=8)
    curr_pos = np.mean(curr_pos)
    print('y: volts:', curr_pos, 'um', volts_to_microm(curr_pos, 'y'))

Old y: volts: 0.0466367582859122 um 0.2887609987739768
y: volts: 0.149468438858534 um 0.9254643177678823


In [ ]:
# scan_size = 10   #um
# mesh_size = 100  #number of points per line scan
# scan_rate = 100  #Hz
# terminal = "/Dev1/PFI1"

pts = np.linspace(-scan_size/2, scan_size/2, 100)

for i in pts:
    for j in pts:
        # move to position i, j
        # read position
        # read APD counts
        # append new data and plot